# Resumo do código

### Código que gera o ficheiro: OUT2_DataFusion
---
O objectivo é receber dados dos Ninjas e da Delta e devolver um ficheiro Wide que permita verificar o comportamento de vários produtos em diversas lojas através de representações gráficas. 

Devolve um ficheiro que entra no R para gerar o Dashboard.

---
- Inputs

> __Dados dos ninjas__ em formato Wide
> - 1's e 0's consoante a presença e ausência do produto, Data e Loja

> __Dados da Delta__ em formato Wide
> - Stocks e trânsito, Sellout dia anterior

- Outputs

> __Ficheiro Wide__ 

> __Métricas novas:__
> - Roturas de Stock, Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - STK
> - (Novo) Balanço
> - (Novo) Dias para a rotura de stock e de prateleira


In [1]:
import pandas as pd
import numpy as np

def escrever_excel(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)

# Ler e Mudar nomes

- # `Ninjas (sinal)`

In [2]:
#Info Ninja
dfNinjas1=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas1fds.xlsx")
dfNinjas2=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas2fds.xlsx")
dfNinjas3=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas3fds.xlsx")
dfNinjas4=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\1Ninjas4fds.xlsx")

# Juntar dataframes no caso de serem vários ficheiros
dataframes=[dfNinjas1,dfNinjas2,dfNinjas3,dfNinjas4]
dfNinjas=pd.concat(dataframes, ignore_index=True)

#Renomear colunas
dfNinjas = dfNinjas.rename(columns={' Data de Resposta': 'DATA', "Código da loja":"STORE", "Nome da Loja":"Loja"})

> Definir variáveis importantes
> - Se houver um ficheiro com os produtos

In [3]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
resto = dfNinjas.columns.difference(produtos)

> - Se se quiser usar o primeiro e o último produtos

In [ ]:
# Nomes de entrada
#x= input("Primeiro produto: ")                # CAFÉ DELTA Q MYTHIQ 80CAP
#y= input("Último produto: ")                  # BEBIDA CEREAIS DELTA C/20%CAFE FR 200G

# Passar para uma lista
#produtos = dfNinjas.columns[dfNinjas.columns.get_loc(x): dfNinjas.columns.get_loc(y)+1].tolist()
#resto = dfNinjas.columns.difference(produtos)

---

## <font color=red> Base Modelo </font>

# Estruturar a base de dados

- altura do dia

In [5]:
# Hora para datetime
dfNinjas['Hora']= pd.to_datetime(dfNinjas['Hora'], format='%H:%M:%S').dt.time  

#coluna nova para data e hora
dfNinjas['DataComp'] = pd.to_datetime(dfNinjas['DATA'].dt.strftime('%Y-%m-%d') + ' ' + dfNinjas['Hora'].astype(str)) 

In [6]:
#Definir que altura do dia é o quê
altura_do_dia = lambda x: 'Manhã' if x < 12 else ('Tarde' if x < 18 else 'Noite')

dfNinjas['Altura_do_Dia'] = dfNinjas['DataComp'].apply(lambda x: altura_do_dia(x.hour))
#dfNinjas['Altura do Dia'] = dfNinjas['DATA'].apply(lambda x: altura_do_dia(x.hour))
dfNinjas=dfNinjas.drop(columns=["DataComp"])

# Fazer o dataframe que servirá de modelo

In [7]:
import itertools

# Definir os valores para cada coluna
dates = dfNinjas["DATA"].unique()
stores = dfNinjas["STORE"].unique()

# Fazer todas as combinações de valores
combinations = list(itertools.product(dates, stores))

# Criar dataframe com essas combinações
dfModelo = pd.DataFrame(combinations, columns=['DATA', 'STORE'])

# Converter para Datetime
dfModelo['DATA'] = pd.to_datetime(dfModelo['DATA'], format='%Y-%m-%d')

In [8]:

# Definir os valores de altura do dia
times = ['Manhã', 'Tarde', 'Noite']

# Create a list of dataframes, each with a single value from times in a new column
dfs = [dfModelo.assign(Altura_do_Dia=t) for t in times]

# Concatenate the list of dataframes into a single dataframe
df_Mergir = pd.concat(dfs, ignore_index=True)


> `MERGIR`

In [9]:
dfNinjas = pd.merge(df_Mergir, dfNinjas, on=['DATA', 'STORE', 'Altura_do_Dia'], how='left')

dfNinjas = dfNinjas.sort_values(by=["STORE","DATA"])

In [10]:
# Valores nulos no nome da loja vão ser substituidos pelo primeiro valor anterior não-nulo

dfNinjas['Loja'] = dfNinjas['Loja'].fillna(method='ffill')

- Dia

In [11]:
nome_dia = dfNinjas['DATA'].dt.day_name().map({'Friday': 'Sexta', 'Saturday': 'Sábado', 'Sunday': 'Domingo'})

dfNinjas['Dia'] = nome_dia

- Semana

In [12]:
dfNinjas['Semana'] = dfNinjas['DATA'].dt.isocalendar().week

dfNinjas['Semana'] = dfNinjas.groupby('Semana').ngroup() + 1

- Completar

In [13]:
df_columns = dfNinjas.columns.tolist()
indexed_columns = [f"{index}: {column}" for index, column in enumerate(df_columns)]

In [14]:
dfNinjasOrg=dfNinjas.copy()
# Get the current column list
cols = dfNinjasOrg.columns.tolist()

# Remove the last two columns and save them
last_two_cols = cols[-2:]
cols = cols[:-2]

# Insert the last two columns to the desired index
cols.insert(5, last_two_cols[0])
cols.insert(6, last_two_cols[1])

# Reorder the columns in the DataFrame
dfNinjasOrg = dfNinjasOrg[cols]

In [15]:
df_columns = dfNinjasOrg.columns.tolist()
indexed_columns = [f"{index}: {column}" for index, column in enumerate(df_columns)]

## <font color=green> Fim base </font>

---

In [20]:
dfNinjas.head(20)

,DATA,STORE,Altura_do_Dia,Loja,Hora,CAFÉ DELTA Q MYTHIQ 80CAP,CAFÉ DELTA Q QALIDUS 80CAP,CAFÉ DELTA Q QALIDUS 10CAP,CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,CAFÉ BELLISSIMO INTENSO 200GR,CAFÉ DELTA RITUAL MU 220G,CAFÉ SOLÚVEL DELTA FRASCO 100G,CAFÉ SOLÚVEL DELTA FRASCO 200G,CEVADA SOLÚVEL DELTA FRASCO 200G,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Dia,Semana
0,2023-04-14,1,Manhã,Continente Matosinhos,10:39:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Sexta,1
120,2023-04-14,1,Tarde,Continente Matosinhos,14:46:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Sexta,1
241,2023-04-14,1,Noite,Continente Matosinhos,19:08:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Sexta,1
10,2023-04-15,1,Manhã,Continente Matosinhos,10:56:25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Sábado,1
130,2023-04-15,1,Tarde,Continente Matosinhos,15:19:35,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Sábado,1
251,2023-04-15,1,Noite,Continente Matosinhos,19:04:28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Sábado,1
20,2023-04-16,1,Manhã,Continente Matosinhos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Domingo,1
140,2023-04-16,1,Tarde,Continente Matosinhos,15:02:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Domingo,1
261,2023-04-16,1,Noite,Continente Matosinhos,19:20:55,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Domingo,1
30,2023-04-21,1,Manhã,Continente Matosinhos,09:30:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Sexta,2


> Reposição

In [16]:
# Ler ficheiro
dfRepos=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\LojasReposição.xlsx")

# Criar coluna de reposição
dfNinjasOrg['Reposição'] = [1 if val in dfRepos['Loja'].values else 0 for val in dfNinjasOrg['Loja']]

- #  `Delta (stocks, reposição e sellouts)`

> Stocks

In [17]:
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta=pd.read_excel("D:\\B&N Dados\\Delta\\Piloto\\2_Delta.xlsx", sheet_name="Sheet1")

# Renomear
dfDelta = dfDelta.drop(columns=["STORE_NAME","Semana","Dia"])

In [18]:
dfDelta[dfDelta["STOCK"]==0]

,DATA,EAN,DESC_ARTIGO,STORE,STOCK,INTRANSIT,EXPECTED,PRES_STOCK,SELLOUT,SELLOUT_1_Dia_Antes,...,MSA20,CICLOS,Dias_para_Rotura_Stock,Dias_Duração_Linear,Adequação,Balance,Balance_Optimized,Balance_Mediano,Balance_Liberal,Balance_Conservador
142,2023-05-06,5601082049946,CAFÉ BELLISSIMO INTENSO 200GR,2,0,128,0,35,0,8,...,14.2,0.0,0.0,1.750000,Stock Insuf c Forn Adequado,0.156250,0.156250,0.140625,0.0625,0.304688
1023,2023-04-21,5601082053561,CAFÉ DELTA RITUAL MU 220G,223,0,0,32,17,0,14,...,4.0,0.0,0.0,4.358974,Stock Insuf c Forn Adequado,0.121875,0.125000,0.031250,0.0000,0.437500
1024,2023-04-22,5601082053561,CAFÉ DELTA RITUAL MU 220G,223,0,0,32,17,0,0,...,3.1,0.0,0.0,4.358974,Stock Insuf c Forn Adequado,0.121875,0.096875,0.031250,0.0000,0.437500


# VER

In [153]:
#dfNinjas.head()
#dfDelta.info()

# Código para Delta (Stocks)

In [37]:
# Dataframe mergido no fim
dfMeio=dfNinjasOrg.copy()

# Definir os dados que queremos manter da base da Delta
lista_a_manter= dfDelta.drop(columns=["DESC_ARTIGO","EAN"]).columns.tolist()

column_mapping = {
    "STOCK": "STOCK %s",
    "PRES_STOCK": "PRESLINEAR %s",
    "INTRANSIT": "INTRANSIT %s",
    "EXPECTED": "EXPECTED %s",
    "SELLOUT": "SELLOUT %s",
    "SELLOUT_1_Dia_Antes": "SELLOUT_1_Dia_Antes %s",
    'ROTURA': "ROTURA %s",
     'PRE_ROTURA': "PRE_ROTURA %s",
     'STK': "STK %s",
     'MSA10': "MSA10 %s",
     'MSA20': "MSA20 %s",
     'CICLOS': "CICLOS %s",
     'Dias_para_Rotura_Stock': "Dias_para_Rotura_Stock %s",
     'Dias_Duração_Linear': "Dias_Duração_Linear %s",
     'Adequação': "Adequação %s",
     'Balance': "Balance %s",
     'Balance_Optimized': "Balance_Optimized %s",
     'Balance_Mediano': "Balance_Mediano %s",
     'Balance_Liberal': "Balance_Liberal %s",
     'Balance_Conservador': "Balance_Conservador %s",
}

for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta[dfDelta["DESC_ARTIGO"]==coluna][lista_a_manter]   #Dados Ninjas, seleccionar colunas do café específico
    
    #Juntar dados de acordo com o dia e a loja
    dfMeio=pd.merge(dfMeio, dfStocks, how="left", on = ["DATA","STORE"])  
    
    #Mudar nomes para ser adaptado a cada produto
    
    merged_columns = {key: value % coluna for key, value in column_mapping.items()}
    dfMeio = dfMeio.rename(columns=merged_columns)

In [39]:
dfMeio.columns.tolist()

['DATA',
 'STORE',
 'Altura_do_Dia',
 'Loja',
 'Hora',
 'Dia',
 'Semana',
 'CAFÉ DELTA Q MYTHIQ 80CAP',
 'CAFÉ DELTA Q QALIDUS 80CAP',
 'CAFÉ DELTA Q QALIDUS 10CAP',
 'CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G',
 'CAFÉ BELLISSIMO INTENSO 200GR',
 'CAFÉ DELTA RITUAL MU 220G',
 'CAFÉ SOLÚVEL DELTA FRASCO 100G',
 'CAFÉ SOLÚVEL DELTA FRASCO 200G',
 'CEVADA SOLÚVEL DELTA FRASCO 200G',
 'BEBIDA CEREAIS DELTA C/20%CAFE FR 200G',
 'Reposição',
 'STOCK CAFÉ DELTA Q MYTHIQ 80CAP',
 'INTRANSIT CAFÉ DELTA Q MYTHIQ 80CAP',
 'EXPECTED CAFÉ DELTA Q MYTHIQ 80CAP',
 'PRESLINEAR CAFÉ DELTA Q MYTHIQ 80CAP',
 'SELLOUT CAFÉ DELTA Q MYTHIQ 80CAP',
 'SELLOUT_1_Dia_Antes CAFÉ DELTA Q MYTHIQ 80CAP',
 'ROTURA CAFÉ DELTA Q MYTHIQ 80CAP',
 'PRE_ROTURA CAFÉ DELTA Q MYTHIQ 80CAP',
 'STK CAFÉ DELTA Q MYTHIQ 80CAP',
 'MSA10 CAFÉ DELTA Q MYTHIQ 80CAP',
 'MSA20 CAFÉ DELTA Q MYTHIQ 80CAP',
 'CICLOS CAFÉ DELTA Q MYTHIQ 80CAP',
 'Dias_para_Rotura_Stock CAFÉ DELTA Q MYTHIQ 80CAP',
 'Dias_Duração_Linear CAFÉ DELTA Q MYTHIQ 80CA

- Quantos produtos estão presentes? <span style="color:red">Vale a pena? </span>

In [23]:
# Criar coluna com o número dos produtos em causa presentes no instante

dfMeio["Num_Produtos"] = dfMeio[produtos].sum(axis=1)

dfMeio.loc[(dfMeio["Num_Produtos"] == 0) & (dfMeio[produtos[1]].isna()), "Num_Produtos"] = np.nan

In [24]:
dfFinal=dfMeio.copy()

In [25]:
dfFinal.columns

Index(['DATA', 'STORE', 'Altura_do_Dia', 'Loja', 'Hora', 'Dia', 'Semana',
       'CAFÉ DELTA Q MYTHIQ 80CAP', 'CAFÉ DELTA Q QALIDUS 80CAP',
       'CAFÉ DELTA Q QALIDUS 10CAP', 'CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G',
       'CAFÉ BELLISSIMO INTENSO 200GR', 'CAFÉ DELTA RITUAL MU 220G',
       'CAFÉ SOLÚVEL DELTA FRASCO 100G', 'CAFÉ SOLÚVEL DELTA FRASCO 200G',
       'CEVADA SOLÚVEL DELTA FRASCO 200G',
       'BEBIDA CEREAIS DELTA C/20%CAFE FR 200G', 'Reposição',
       'STOCK CAFÉ DELTA Q MYTHIQ 80CAP',
       'PRESLINEAR CAFÉ DELTA Q MYTHIQ 80CAP',
       'INTRANSIT CAFÉ DELTA Q MYTHIQ 80CAP',
       'EXPECTED CAFÉ DELTA Q MYTHIQ 80CAP',
       'SELLOUT CAFÉ DELTA Q MYTHIQ 80CAP', 'STOCK CAFÉ DELTA Q QALIDUS 80CAP',
       'PRESLINEAR CAFÉ DELTA Q QALIDUS 80CAP',
       'INTRANSIT CAFÉ DELTA Q QALIDUS 80CAP',
       'EXPECTED CAFÉ DELTA Q QALIDUS 80CAP',
       'SELLOUT CAFÉ DELTA Q QALIDUS 80CAP',
       'STOCK CAFÉ DELTA Q QALIDUS 10CAP',
       'PRESLINEAR CAFÉ DELTA Q QALIDUS 10CA

# Já temos DataFrame com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição

## Fazer:

- Rotura
- Sinal -> Avaliação da informação ninja vs stock
- Ciclos
- Adequação de Stock

> STK

In [22]:
for i in produtos:

    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    
    stk = "STK %s" %i
      
    dfFinal[stk] = dfFinal[stock] + dfFinal[esperado] + dfFinal[transito]

> Balanço

In [ ]:
for i in produtos:
    stk = "STK %s" %i
    msa = "MSA %s" %i
    
    balance = "BALANCE %s" %i
    
    dfFinal[balance] = dfFinal[msa] / dfFinal[stk]
    
    

> Rotura

In [23]:
for i in produtos:
    rotura  = "ROTURA %s" % i
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    
    dfFinal[rotura] = np.where((dfFinal[stock] > 0) & ~(dfFinal[i].isna()), 0, 
                      np.where((dfFinal[stock] <= 0) & (dfFinal[preslinear] > 0) & ~(dfFinal[i].isna()), 1, 
                      np.nan))


> Pré-Rotura

In [24]:
for i in produtos:
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    pre_rotura= "PRÉ-ROTURA %s" % i
    
    dfFinal[pre_rotura] = (dfFinal[stock] < dfFinal[preslinear]).astype(int)

>Sinal

In [25]:
#dfFinal["Sinal"]= pd.Series(dtype='int')

# Define the mappings of numbers to strings
mapping = {1: "Verdadeiramente Correcto",
           2: "Falso Positivo",
           3: "Falso Negativo",
           4: "Rejeição Correcta"}

for i in produtos:
    sinal = "SINAL %s" % i
    stock  = "STOCK %s" % i
    
    dfFinal[sinal] = np.where((dfFinal[stock] > 0) & (dfFinal[i] == 1), 1,
                     np.where((dfFinal[stock] > 0) & (dfFinal[i] == 0), 2,
                     np.where((dfFinal[stock] <= 0) & (dfFinal[i] == 1), 3,
                     np.where((dfFinal[stock] <= 0) & (dfFinal[i] == 0), 4, 
                     np.nan))))


    # Map the numbers to the corresponding strings
    dfFinal[sinal] = dfFinal[sinal].map(mapping)
    

# 1: Há Stock e Sinal Positivo Ninja "Verdadeiramente Correcto"
# 2: Há Stock e Sinal Negativo Ninja "Falso Positivo"
# 3: Não há Stock e Sinal Positivo Ninja "Falso Negativo"
# 4: Não há Stock e Sinal Negativo Ninja "Rejeição Correcta"

> Ciclos

In [26]:
for i in produtos:
    ciclos = "CICLO %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    dfFinal[ciclos]=dfFinal[stock]/dfFinal[linear]

> Adequação

In [27]:
# Define the mappings of numbers to strings
mapping = {1: "Stock Suficiente",
           2: "Stock Insuf c Forn Adequado",
           3: "Stock Insuf c Forn Desadequado"}

for i in produtos:
    adequa = "ADEQUAÇÃO %s" % i
    ciclos = "CICLO %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    
    dfFinal[adequa] = np.where(dfFinal[ciclos] > 1.1, 1,
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] >= dfFinal[linear]), 2,
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] < dfFinal[linear]), 3, 
                      np.nan)))
    
    # Map the numbers to the corresponding strings
    dfFinal[adequa] = dfFinal[adequa].map(mapping)

In [28]:
dfFinal[dfFinal["CAFÉ DELTA Q MYTHIQ 80CAP"].isna()]

,DATA,STORE,Altura_do_Dia,Loja,Hora,Dia,Semana,CAFÉ DELTA Q MYTHIQ 80CAP,CAFÉ DELTA Q QALIDUS 80CAP,CAFÉ DELTA Q QALIDUS 10CAP,CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,CAFÉ BELLISSIMO INTENSO 200GR,CAFÉ DELTA RITUAL MU 220G,CAFÉ SOLÚVEL DELTA FRASCO 100G,CAFÉ SOLÚVEL DELTA FRASCO 200G,CEVADA SOLÚVEL DELTA FRASCO 200G,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,STOCK CAFÉ DELTA Q MYTHIQ 80CAP,PRESLINEAR CAFÉ DELTA Q MYTHIQ 80CAP,INTRANSIT CAFÉ DELTA Q MYTHIQ 80CAP,EXPECTED CAFÉ DELTA Q MYTHIQ 80CAP,SELLOUT CAFÉ DELTA Q MYTHIQ 80CAP,STOCK CAFÉ DELTA Q QALIDUS 80CAP,PRESLINEAR CAFÉ DELTA Q QALIDUS 80CAP,INTRANSIT CAFÉ DELTA Q QALIDUS 80CAP,EXPECTED CAFÉ DELTA Q QALIDUS 80CAP,SELLOUT CAFÉ DELTA Q QALIDUS 80CAP,STOCK CAFÉ DELTA Q QALIDUS 10CAP,PRESLINEAR CAFÉ DELTA Q QALIDUS 10CAP,INTRANSIT CAFÉ DELTA Q QALIDUS 10CAP,EXPECTED CAFÉ DELTA Q QALIDUS 10CAP,SELLOUT CAFÉ DELTA Q QALIDUS 10CAP,STOCK CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,PRESLINEAR CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,INTRANSIT CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,EXPECTED CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,SELLOUT CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,STOCK CAFÉ BELLISSIMO INTENSO 200GR,PRESLINEAR CAFÉ BELLISSIMO INTENSO 200GR,INTRANSIT CAFÉ BELLISSIMO INTENSO 200GR,EXPECTED CAFÉ BELLISSIMO INTENSO 200GR,SELLOUT CAFÉ BELLISSIMO INTENSO 200GR,STOCK CAFÉ DELTA RITUAL MU 220G,PRESLINEAR CAFÉ DELTA RITUAL MU 220G,INTRANSIT CAFÉ DELTA RITUAL MU 220G,EXPECTED CAFÉ DELTA RITUAL MU 220G,SELLOUT CAFÉ DELTA RITUAL MU 220G,STOCK CAFÉ SOLÚVEL DELTA FRASCO 100G,PRESLINEAR CAFÉ SOLÚVEL DELTA FRASCO 100G,INTRANSIT CAFÉ SOLÚVEL DELTA FRASCO 100G,EXPECTED CAFÉ SOLÚVEL DELTA FRASCO 100G,SELLOUT CAFÉ SOLÚVEL DELTA FRASCO 100G,STOCK CAFÉ SOLÚVEL DELTA FRASCO 200G,PRESLINEAR CAFÉ SOLÚVEL DELTA FRASCO 200G,INTRANSIT CAFÉ SOLÚVEL DELTA FRASCO 200G,EXPECTED CAFÉ SOLÚVEL DELTA FRASCO 200G,SELLOUT CAFÉ SOLÚVEL DELTA FRASCO 200G,STOCK CEVADA SOLÚVEL DELTA FRASCO 200G,PRESLINEAR CEVADA SOLÚVEL DELTA FRASCO 200G,INTRANSIT CEVADA SOLÚVEL DELTA FRASCO 200G,EXPECTED CEVADA SOLÚVEL DELTA FRASCO 200G,SELLOUT CEVADA SOLÚVEL DELTA FRASCO 200G,STOCK BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,PRESLINEAR BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,INTRANSIT BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,EXPECTED BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,SELLOUT BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Num_Produtos,ROTURA CAFÉ DELTA Q MYTHIQ 80CAP,ROTURA CAFÉ DELTA Q QALIDUS 80CAP,ROTURA CAFÉ DELTA Q QALIDUS 10CAP,ROTURA CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,ROTURA CAFÉ BELLISSIMO INTENSO 200GR,ROTURA CAFÉ DELTA RITUAL MU 220G,ROTURA CAFÉ SOLÚVEL DELTA FRASCO 100G,ROTURA CAFÉ SOLÚVEL DELTA FRASCO 200G,ROTURA CEVADA SOLÚVEL DELTA FRASCO 200G,ROTURA BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,PRÉ-ROTURA CAFÉ DELTA Q MYTHIQ 80CAP,PRÉ-ROTURA CAFÉ DELTA Q QALIDUS 80CAP,PRÉ-ROTURA CAFÉ DELTA Q QALIDUS 10CAP,PRÉ-ROTURA CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,PRÉ-ROTURA CAFÉ BELLISSIMO INTENSO 200GR,PRÉ-ROTURA CAFÉ DELTA RITUAL MU 220G,PRÉ-ROTURA CAFÉ SOLÚVEL DELTA FRASCO 100G,PRÉ-ROTURA CAFÉ SOLÚVEL DELTA FRASCO 200G,PRÉ-ROTURA CEVADA SOLÚVEL DELTA FRASCO 200G,PRÉ-ROTURA BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,SINAL CAFÉ DELTA Q MYTHIQ 80CAP,SINAL CAFÉ DELTA Q QALIDUS 80CAP,SINAL CAFÉ DELTA Q QALIDUS 10CAP,SINAL CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,SINAL CAFÉ BELLISSIMO INTENSO 200GR,SINAL CAFÉ DELTA RITUAL MU 220G,SINAL CAFÉ SOLÚVEL DELTA FRASCO 100G,SINAL CAFÉ SOLÚVEL DELTA FRASCO 200G,SINAL CEVADA SOLÚVEL DELTA FRASCO 200G,SINAL BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,CICLO CAFÉ DELTA Q MYTHIQ 80CAP,CICLO CAFÉ DELTA Q QALIDUS 80CAP,CICLO CAFÉ DELTA Q QALIDUS 10CAP,CICLO CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,CICLO CAFÉ BELLISSIMO INTENSO 200GR,CICLO CAFÉ DELTA RITUAL MU 220G,CICLO CAFÉ SOLÚVEL DELTA FRASCO 100G,CICLO CAFÉ SOLÚVEL DELTA FRASCO 200G,CICLO CEVADA SOLÚVEL DELTA FRASCO 200G,CICLO BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,ADEQUAÇÃO CAFÉ DELTA Q MYTHIQ 80CAP,ADEQUAÇÃO CAFÉ DELTA Q QALIDUS 80CAP,ADEQUAÇÃO CAFÉ DELTA Q QALIDUS 10CAP,ADEQUAÇÃO 

- Dias para a rotura

In [29]:
for i in produtos:
    stock  = "STOCK %s" % i
    sellout = "SELLOUT %s" % i
    dias_rotura = "Dias_para_Rotura %s" % i
    
    dfFinal[dias_rotura] = dfFinal[stock]/dfFinal[sellout]

- Dias para a rotura de prateleira

In [40]:
for i in produtos:
    linear = "PRESLINEAR %s" % i
    sellout = "SELLOUT %s" % i
    dias_rotura_prat = "Dias_para_Rotura_Linear %s" % i
    
    dfFinal[dias_rotura_prat] = dfFinal[linear] / dfFinal[sellout]

In [41]:
dfFinal.head(20)

,DATA,STORE,Altura_do_Dia,Loja,Hora,Dia,Semana,CAFÉ DELTA Q MYTHIQ 80CAP,CAFÉ DELTA Q QALIDUS 80CAP,CAFÉ DELTA Q QALIDUS 10CAP,CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,CAFÉ BELLISSIMO INTENSO 200GR,CAFÉ DELTA RITUAL MU 220G,CAFÉ SOLÚVEL DELTA FRASCO 100G,CAFÉ SOLÚVEL DELTA FRASCO 200G,CEVADA SOLÚVEL DELTA FRASCO 200G,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,STOCK CAFÉ DELTA Q MYTHIQ 80CAP,PRESLINEAR CAFÉ DELTA Q MYTHIQ 80CAP,INTRANSIT CAFÉ DELTA Q MYTHIQ 80CAP,EXPECTED CAFÉ DELTA Q MYTHIQ 80CAP,SELLOUT CAFÉ DELTA Q MYTHIQ 80CAP,STOCK CAFÉ DELTA Q QALIDUS 80CAP,PRESLINEAR CAFÉ DELTA Q QALIDUS 80CAP,INTRANSIT CAFÉ DELTA Q QALIDUS 80CAP,EXPECTED CAFÉ DELTA Q QALIDUS 80CAP,SELLOUT CAFÉ DELTA Q QALIDUS 80CAP,STOCK CAFÉ DELTA Q QALIDUS 10CAP,PRESLINEAR CAFÉ DELTA Q QALIDUS 10CAP,INTRANSIT CAFÉ DELTA Q QALIDUS 10CAP,EXPECTED CAFÉ DELTA Q QALIDUS 10CAP,SELLOUT CAFÉ DELTA Q QALIDUS 10CAP,STOCK CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,PRESLINEAR CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,INTRANSIT CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,EXPECTED CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,SELLOUT CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,STOCK CAFÉ BELLISSIMO INTENSO 200GR,PRESLINEAR CAFÉ BELLISSIMO INTENSO 200GR,INTRANSIT CAFÉ BELLISSIMO INTENSO 200GR,EXPECTED CAFÉ BELLISSIMO INTENSO 200GR,SELLOUT CAFÉ BELLISSIMO INTENSO 200GR,STOCK CAFÉ DELTA RITUAL MU 220G,PRESLINEAR CAFÉ DELTA RITUAL MU 220G,INTRANSIT CAFÉ DELTA RITUAL MU 220G,EXPECTED CAFÉ DELTA RITUAL MU 220G,SELLOUT CAFÉ DELTA RITUAL MU 220G,STOCK CAFÉ SOLÚVEL DELTA FRASCO 100G,PRESLINEAR CAFÉ SOLÚVEL DELTA FRASCO 100G,INTRANSIT CAFÉ SOLÚVEL DELTA FRASCO 100G,EXPECTED CAFÉ SOLÚVEL DELTA FRASCO 100G,SELLOUT CAFÉ SOLÚVEL DELTA FRASCO 100G,STOCK CAFÉ SOLÚVEL DELTA FRASCO 200G,PRESLINEAR CAFÉ SOLÚVEL DELTA FRASCO 200G,INTRANSIT CAFÉ SOLÚVEL DELTA FRASCO 200G,EXPECTED CAFÉ SOLÚVEL DELTA FRASCO 200G,SELLOUT CAFÉ SOLÚVEL DELTA FRASCO 200G,STOCK CEVADA SOLÚVEL DELTA FRASCO 200G,PRESLINEAR CEVADA SOLÚVEL DELTA FRASCO 200G,INTRANSIT CEVADA SOLÚVEL DELTA FRASCO 200G,EXPECTED CEVADA SOLÚVEL DELTA FRASCO 200G,SELLOUT CEVADA SOLÚVEL DELTA FRASCO 200G,STOCK BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,PRESLINEAR BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,INTRANSIT BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,EXPECTED BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,SELLOUT BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Num_Produtos,ROTURA CAFÉ DELTA Q MYTHIQ 80CAP,ROTURA CAFÉ DELTA Q QALIDUS 80CAP,ROTURA CAFÉ DELTA Q QALIDUS 10CAP,ROTURA CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,ROTURA CAFÉ BELLISSIMO INTENSO 200GR,ROTURA CAFÉ DELTA RITUAL MU 220G,ROTURA CAFÉ SOLÚVEL DELTA FRASCO 100G,ROTURA CAFÉ SOLÚVEL DELTA FRASCO 200G,ROTURA CEVADA SOLÚVEL DELTA FRASCO 200G,ROTURA BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,PRÉ-ROTURA CAFÉ DELTA Q MYTHIQ 80CAP,PRÉ-ROTURA CAFÉ DELTA Q QALIDUS 80CAP,PRÉ-ROTURA CAFÉ DELTA Q QALIDUS 10CAP,PRÉ-ROTURA CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,PRÉ-ROTURA CAFÉ BELLISSIMO INTENSO 200GR,PRÉ-ROTURA CAFÉ DELTA RITUAL MU 220G,PRÉ-ROTURA CAFÉ SOLÚVEL DELTA FRASCO 100G,PRÉ-ROTURA CAFÉ SOLÚVEL DELTA FRASCO 200G,PRÉ-ROTURA CEVADA SOLÚVEL DELTA FRASCO 200G,PRÉ-ROTURA BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,SINAL CAFÉ DELTA Q MYTHIQ 80CAP,SINAL CAFÉ DELTA Q QALIDUS 80CAP,SINAL CAFÉ DELTA Q QALIDUS 10CAP,SINAL CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,SINAL CAFÉ BELLISSIMO INTENSO 200GR,SINAL CAFÉ DELTA RITUAL MU 220G,SINAL CAFÉ SOLÚVEL DELTA FRASCO 100G,SINAL CAFÉ SOLÚVEL DELTA FRASCO 200G,SINAL CEVADA SOLÚVEL DELTA FRASCO 200G,SINAL BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,CICLO CAFÉ DELTA Q MYTHIQ 80CAP,CICLO CAFÉ DELTA Q QALIDUS 80CAP,CICLO CAFÉ DELTA Q QALIDUS 10CAP,CICLO CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,CICLO CAFÉ BELLISSIMO INTENSO 200GR,CICLO CAFÉ DELTA RITUAL MU 220G,CICLO CAFÉ SOLÚVEL DELTA FRASCO 100G,CICLO CAFÉ SOLÚVEL DELTA FRASCO 200G,CICLO CEVADA SOLÚVEL DELTA FRASCO 200G,CICLO BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,ADEQUAÇÃO CAFÉ DELTA Q MYTHIQ 80CAP,ADEQUAÇÃO CAFÉ DELTA Q QALIDUS 80CAP,ADEQUAÇÃO CAFÉ DELTA Q QALIDUS 10CAP,ADEQUAÇÃO 

# Acabamos com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição
- Rotura
- Pré-Rotura
- Sinal 
- Ciclos
- Adequação de Stock
- Dias para a rotura normal
- Dias para a rotura móvel XXXXXXXXXXX
- Dias para a rotura de prateleira
- Balanço

# Escrever

In [34]:
escrever_excel(dfFinal, "OUT2DataFusion2")
#dfFinalCorr.to_csv('ficheiro_NSS.csv', index=False)